# Spatial modelling of probabilities in production: all GB

This document contains the code used to perform spatial modeling of probabilities as deployed for the case of GB. It is the production version of [`sp_model_chip_probabilities`](sp_model_chip_probabilities).

The following models are fitted:

1. `argmax`: pick the top probability from those produced for the chip by the neural net
1. `logit`: fit a Logit model using the chip probabilities from the neural net
1. `logit_wx`: fit a Logit model using the chip probabilities _and_ the spatial lag of chip probabilities (i.g., using also the probabilities from neighboring chips)
1. `gbtree`: fit a histogram-based gradient boosted tree model using the chip probabilities from the neural net
1. `gbtree_wx`: fit a histogram-based gradient boosted tree model using the chip probabilities _and_ the spatial lag of chip probabilities (i.g., using also the probabilities from neighboring chips)

These five models will be fitted to the results from 